In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_excel('../game_scraping/scraping_results.xlsx', index_col=0)

In [4]:
data['Week'].unique()

data['Series'] = 1

# Initialize the series counter
series_counter = 1

# Iterate through the dataframe to update the Series column
for i in range(1, len(data)):
    if data.loc[i, 'Week'] == 1 and data.loc[i-1, 'Week'] != 1:
        series_counter += 1
    data.loc[i, 'Series'] = series_counter

data['revenue_unit'] = data['Price'] * data['Sales']

data['previous_sales'] = data.groupby('Series')['Sales'].shift(1)
data['previous_price'] = data.groupby('Series')['Price'].shift(1)
data['change_price'] = data.groupby('Series')['Price'].diff()
data['remaining_previous'] = data.groupby('Series')['Remaining'].shift(1)
data

,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect,Series,revenue_unit,previous_sales,previous_price,change_price,remaining_previous
0,1,60,87,1913,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1,5220,NaN,NaN,NaN,NaN
1,2,60,74,1839,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1,4440,87.0,60.0,0.0,1913.0
2,3,54,147,1692,"Your revenue: $85,752, Perfect foresight strat...",1,15.8,85752,101892,1,7938,74.0,60.0,-6.0,1839.0
3,4,54,45,1647,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1,2430,147.0,54.0,0.0,1692.0
4,5,48,160,1487,"Your revenue: $85,752, Perfect foresight strat...",2,15.8,85752,101892,1,7680,45.0,54.0,-6.0,1647.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,11,36,100,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860,103596,100,3600,206.0,36.0,0.0,100.0
1496,12,36,0,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860,103596,100,0,100.0,36.0,0.0,0.0
1497,13,36,0,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860,103596,100,0,0.0,36.0,0.0,0.0
1498,14,36,0,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860,103596,100,0,0.0,36.0,0.0,0.0


In [5]:
from sklearn.linear_model import LinearRegression   
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error

linear_model = LinearRegression()
linear_model.fit(data[['Week','Price']], data[['Sales']])
preds = linear_model.predict(data[['Week','Price']])

print('R2 score:',r2_score(data[['Sales']], preds))
print('MAE:',mean_absolute_error(data[['Sales']], preds))
print('MAPE:',mean_absolute_percentage_error(data[['Sales']], preds))   
print(linear_model.coef_)

R2 score: 0.3171610790613203
MAE: 62.95957184179123
MAPE: 6.655982285142552e+16
[[-26.81353903 -12.52288096]]


In [6]:
### More variables and boosting model


from xgboost import XGBRegressor

#### Train and test split

train = data[data['Series'] <= 80]
test = data[data['Series'] > 80]

model = XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=123, max_depth = 3)
model.fit(train[['Week','Price','previous_sales','previous_price','change_price','remaining_previous']], train['Sales'])
preds = model.predict(test[['Week','Price','previous_sales','previous_price','change_price','remaining_previous']])

print('R2 score:',r2_score(test[['Sales']], preds))
print('MAE:',mean_absolute_error(test[['Sales']], preds))
print('MAPE:',mean_absolute_percentage_error(test[['Sales']], preds))


R2 score: 0.7010067543411818
MAE: 34.11835731665293
MAPE: 2929533032948476.5


In [82]:
test.head()

,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect,Series,revenue_unit,previous_sales,previous_price,change_price,remaining_previous
1200,1,60,87,1913,"Your revenue: $84,354, Perfect foresight strat...",0,17.7,84354,102558,81,5220,NaN,NaN,NaN,NaN
1201,2,60,64,1849,"Your revenue: $84,354, Perfect foresight strat...",0,17.7,84354,102558,81,3840,87.0,60.0,0.0,1913.0
1202,3,54,141,1708,"Your revenue: $84,354, Perfect foresight strat...",1,17.7,84354,102558,81,7614,64.0,60.0,-6.0,1849.0
1203,4,54,132,1576,"Your revenue: $84,354, Perfect foresight strat...",0,17.7,84354,102558,81,7128,141.0,54.0,0.0,1708.0
1204,5,48,200,1376,"Your revenue: $84,354, Perfect foresight strat...",2,17.7,84354,102558,81,9600,132.0,54.0,-6.0,1576.0


In [456]:
from gurobipy import Model, GRB
import gurobipy as gp


pricing_model = Model('pricing')

### Decision variables 

prices_decision = pricing_model.addVars(data['Week'].unique(), lb=0, ub = 3, vtype=GRB.INTEGER, name='prices')

### Objective function
def obj(decisions):
    ## Decisions is a list of len = 15
    initial_price = 60
    allowed_prices = [60, 54, 48, 36]
    initial_stock = 2000

    prices = []
    stock_evolution = []
    revenue_evolution = []
    sales_evolution = []    
    for week in range(1,16):
        print(decisions)
        decision = decisions[week].getAttr('x')
        if week == 1: 
            tobe_predicted = np.array([[week, initial_price, np.nan, np.nan, np.nan, np.nan]])
            sales_predicted = np.floor(model.predict(tobe_predicted))
            new_remaining = initial_stock - sales_predicted[0]
            stock_evolution.append(new_remaining)
            old_price = initial_price
            prices.append(initial_price)
            revenue = sales_predicted[0] * initial_price
            revenue_evolution.append(revenue)
            sales_evolution.append(sales_predicted[0])
            
        else:
            new_price = allowed_prices[decision]
            
            if new_price <= old_price:
                new_price = new_price
            else:
                new_price = old_price
            tobe_predicted = np.array([[week, new_price, sales_predicted[0], old_price, new_price - old_price, new_remaining]])
            sales_predicted = np.floor(model.predict(tobe_predicted))
            if sales_predicted[0] > new_remaining:
                sales_predicted[0] = new_remaining
            new_remaining = new_remaining - sales_predicted[0]
            revenue = sales_predicted[0] * new_price
            sales_evolution.append(sales_predicted[0])
            revenue_evolution.append(revenue)
            stock_evolution.append(new_remaining)
            old_price = new_price
            prices.append(new_price)

    total_revenue = sum(revenue_evolution)
    return total_revenue

pricing_model.setObjective(obj(prices_decision), GRB.MAXIMIZE)

pricing_model.optimize()

AttributeError: Index out of range for attribute 'X'

In [21]:
def obj(decisions):
    ## Decisions is a list of len = 15
    initial_price = 60
    allowed_prices = [60, 54, 48, 36]
    initial_stock = 2000

    prices = []
    stock_evolution = []
    revenue_evolution = []
    sales_evolution = []    
    for week in range(1,16):
        decision = decisions[week-1]
        if week == 1: 
            tobe_predicted = np.array([[week, initial_price, np.nan, np.nan, np.nan, np.nan]])
            sales_predicted = np.floor(model.predict(tobe_predicted))
            new_remaining = initial_stock - sales_predicted[0]
            stock_evolution.append(new_remaining)
            old_price = initial_price
            prices.append(initial_price)
            revenue = sales_predicted[0] * initial_price
            revenue_evolution.append(revenue)
            sales_evolution.append(sales_predicted[0])
            
        else:
            new_price = allowed_prices[decision]
            
            if new_price <= old_price:
                new_price = new_price
            else:
                new_price = old_price
            tobe_predicted = np.array([[week, new_price, sales_predicted[0], old_price, new_price - old_price, new_remaining]])
            sales_predicted = np.floor(model.predict(tobe_predicted))
            if sales_predicted[0] > new_remaining:
                sales_predicted[0] = new_remaining
            new_remaining = new_remaining - sales_predicted[0]
            revenue = sales_predicted[0] * new_price
            sales_evolution.append(sales_predicted[0])
            revenue_evolution.append(revenue)
            stock_evolution.append(new_remaining)
            old_price = new_price
            prices.append(new_price)

    total_revenue = sum(revenue_evolution)
    return total_revenue,prices

In [22]:
import optuna

def objective(trial):
    prices = [
        trial.suggest_int('price_1', 0, 3),
        trial.suggest_int('price_2', 0, 3),
        trial.suggest_int('price_3', 0, 3),
        trial.suggest_int('price_4', 0, 3),
        trial.suggest_int('price_5', 0, 3),
        trial.suggest_int('price_6', 0, 3),
        trial.suggest_int('price_7', 0, 3),
        trial.suggest_int('price_8', 0, 3),
        trial.suggest_int('price_9', 0, 3),
        trial.suggest_int('price_10', 0, 3),
        trial.suggest_int('price_11', 0, 3),
        trial.suggest_int('price_12', 0, 3),
        trial.suggest_int('price_13', 0, 3),
        trial.suggest_int('price_14', 0, 3),
        trial.suggest_int('price_15', 0, 3),
    ]
    total_revenue,prices = obj(prices)
              
    return total_revenue

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-05-20 18:39:01,485] A new study created in memory with name: no-name-bc914d08-0a3c-459e-8203-e5efe2f936fd
[I 2024-05-20 18:39:01,490] Trial 0 finished with value: 83688.0 and parameters: {'price_1': 3, 'price_2': 1, 'price_3': 2, 'price_4': 1, 'price_5': 1, 'price_6': 0, 'price_7': 3, 'price_8': 0, 'price_9': 0, 'price_10': 1, 'price_11': 1, 'price_12': 0, 'price_13': 0, 'price_14': 3, 'price_15': 2}. Best is trial 0 with value: 83688.0.
[I 2024-05-20 18:39:01,494] Trial 1 finished with value: 74112.0 and parameters: {'price_1': 1, 'price_2': 3, 'price_3': 2, 'price_4': 2, 'price_5': 0, 'price_6': 2, 'price_7': 1, 'price_8': 0, 'price_9': 1, 'price_10': 0, 'price_11': 3, 'price_12': 0, 'price_13': 0, 'price_14': 0, 'price_15': 1}. Best is trial 0 with value: 83688.0.
[I 2024-05-20 18:39:01,499] Trial 2 finished with value: 77700.0 and parameters: {'price_1': 1, 'price_2': 2, 'price_3': 2, 'price_4': 3, 'price_5': 1, 'price_6': 0, 'price_7': 1, 'price_8': 2, 'price_9': 1, 'price

In [27]:
obj(list(study.best_params.values()))[1]

[60, 54, 54, 54, 54, 54, 54, 54, 48, 48, 48, 48, 48, 48, 48]

### Evaluation of Model

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import time

def convert_pricing_strategy_to_actions(strategy):
    price_to_action = {
        60: 'maintain',  # Assuming $60 is the full price hence 'maintain'
        54: 'ten',       # 10% discount action
        48: 'twenty',    # 20% discount action
        36: 'forty'      # 40% discount action
    }

    actions = [price_to_action[strategy[0]]]  # Start with the initial price action
    last_price = strategy[0]

    for price in strategy[1:]:  # Skip the first price since it's already added
        if price != last_price:
            actions.append(price_to_action[price])  # Append new price action
        actions.append('maintain')  # Always append 'maintain' after any action
        last_price = price
    
    actions = actions[1:]
    
    # Ensure the actions list is exactly 14 weeks long
    if len(actions) > 14:
        actions = actions[:14]
    elif len(actions) < 14:
        actions.extend(['maintain'] * (14 - len(actions)))  # Pad with 'maintain' if less than 14

    return actions

def init_driver():
    options = Options()
    options.add_argument("--start-maximized")  # Opens the browser window maximized
    chrome_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=chrome_service, options=options)
    return driver

strategy = [60, 54, 54, 54, 54, 54, 54, 54, 48, 48, 48, 48, 48, 48, 48]
actions = convert_pricing_strategy_to_actions(strategy)

# Initialize the driver and navigate to the game
driver = init_driver()
driver.get("http://www.randhawa.us/games/retailer/nyu.html")
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "result-table")))

# Assume strategy_df is already defined and contains 'Actions' column as discussed
all_runs = []
summary_data = []  # To store summary of each run
attempts = 100

for attempt in range(attempts):
    print(f"Attempt {attempt+1}/{attempts}")
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "practiceButton"))).click()
    time.sleep(1)  # Ensure the game has reset

    # Fetch initial demand and find closest strategy
    first_sales_value = int(WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//table[@id='result-table']/thead/tr[2]/td[3]"))
    ).text)
    
    buttons = {
        "maintain": driver.find_element(By.ID, "maintainButton"),
        "ten": driver.find_element(By.ID, "tenButton"),
        "twenty": driver.find_element(By.ID, "twentyButton"),
        "forty": driver.find_element(By.ID, "fortyButton")
    }
    
    # Execute strategy and capture data
    for i, action in enumerate(actions):
        buttons[action].click()
        time.sleep(1)  # Allow time for the table to update

    # Capture data after last action
    text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "result-table"))).text
    lines = text.split('\n')
    data_lines = lines[1:]  # adjust the index based on where you want to start
    data = [line.split() for line in data_lines]
    df = pd.DataFrame(data, columns=['Week', 'Price', 'Sales', 'Remaining Inventory'])
    all_runs.append(df)
    
    # At the end of 15 weeks, capture the final financial results
    revenue = driver.find_element(By.ID, "rev").text
    perfect_foresight = driver.find_element(By.ID, "perfect").text
    percentage = driver.find_element(By.ID, "percentage").text
    # Strip leading comma and space, then split by space
    percentage_value = percentage.strip(", ").split(" ")[1].strip('%')
    
    summary_data.append({
        'First Sales Value': first_sales_value,
        'Revenue': revenue,
        'Perfect Foresight Strategy': perfect_foresight,
        'Percentage': percentage_value
    })

driver.quit()

all_data = pd.concat(all_runs, ignore_index=True)  # Concatenate all DataFrame objects in the list
summary_df = pd.DataFrame(summary_data)

print(f"Total rows captured over all attempts: {len(all_data)}")

Attempt 1/100
Attempt 2/100
Attempt 3/100
Attempt 4/100
Attempt 5/100
Attempt 6/100
Attempt 7/100
Attempt 8/100
Attempt 9/100
Attempt 10/100
Attempt 11/100
Attempt 12/100
Attempt 13/100
Attempt 14/100
Attempt 15/100
Attempt 16/100
Attempt 17/100
Attempt 18/100
Attempt 19/100
Attempt 20/100
Attempt 21/100
Attempt 22/100
Attempt 23/100
Attempt 24/100
Attempt 25/100
Attempt 26/100
Attempt 27/100
Attempt 28/100
Attempt 29/100
Attempt 30/100
Attempt 31/100
Attempt 32/100
Attempt 33/100
Attempt 34/100
Attempt 35/100
Attempt 36/100
Attempt 37/100
Attempt 38/100
Attempt 39/100
Attempt 40/100
Attempt 41/100
Attempt 42/100
Attempt 43/100
Attempt 44/100
Attempt 45/100
Attempt 46/100
Attempt 47/100
Attempt 48/100
Attempt 49/100
Attempt 50/100
Attempt 51/100
Attempt 52/100
Attempt 53/100
Attempt 54/100
Attempt 55/100
Attempt 56/100
Attempt 57/100
Attempt 58/100
Attempt 59/100
Attempt 60/100
Attempt 61/100
Attempt 62/100
Attempt 63/100
Attempt 64/100
Attempt 65/100
Attempt 66/100
Attempt 67/100
Atte

In [21]:
# List of specific columns to modify
columns_to_modify = ['Revenue', 'Perfect Foresight Strategy','Percentage']

# Strip commas and convert to integers
for column in columns_to_modify:
    summary_df[column] = summary_df[column].astype(str).str.replace(',', '').str.replace('$', '').astype(float)
            
summary_df.dtypes

First Sales Value               int64
Revenue                       float64
Perfect Foresight Strategy    float64
Percentage                    float64
dtype: object

In [22]:
summary_df

,First Sales Value,Revenue,Perfect Foresight Strategy,Percentage
0,105,103926.0,111804.0,7.0
1,122,105018.0,114222.0,8.1
2,76,82170.0,91248.0,9.9
3,131,105576.0,120000.0,12.0
4,108,102630.0,105714.0,2.9
...,...,...,...,...
95,90,103164.0,106434.0,3.1
96,107,103980.0,109344.0,4.9
97,89,101940.0,101940.0,0.0
98,88,96870.0,100452.0,3.6


In [23]:
summary_df.describe()

,First Sales Value,Revenue,Perfect Foresight Strategy,Percentage
count,100.000000,100.000000,100.000000,100.00000
mean,93.400000,93726.540000,101321.460000,8.05600
std,25.871087,17459.909176,16413.938868,5.69207
min,15.000000,16716.000000,22032.000000,0.00000
25%,76.750000,86652.000000,95025.000000,3.67500
50%,91.000000,102834.000000,104547.000000,7.90000
75%,108.500000,104442.000000,111490.500000,10.82500
max,155.000000,107430.000000,120000.000000,24.20000
